In [ ]:
# Notebook for generating descriptions of new cell types not identified in EM from info in fig S2
# see https://github.com/FlyBase/drosophila-anatomy-developmental-ontology/issues/2073

In [ ]:
import pandas as pd
import csv
import re
from num2words import num2words
from collections import OrderedDict

# Input data
new_terms = pd.read_csv('new_types.tsv', sep='\t', dtype='str')
region_ids = pd.read_csv('region_fbbt.tsv', sep='\t', dtype='str', index_col='region')
data = pd.read_csv('neuron_regions.tsv', sep='\t', dtype='str')

#data[:10]

In [ ]:
# Make a dictionary with key - column header & value = template specification (first row of table).

template_seed = OrderedDict([ ('ID' , 'ID'), ("Name" , "A rdfs:label"), ("Definition" , "A IAO:0000115"),\
                      ("Xref_def" , ">A oboInOwl:hasDbXref"),\
                      ("created_by" , "AI dc:contributor"),\
                      ("creation_date", "AT dc:date^^xsd:dateTime"), ("comment" , "A rdfs:comment")])

# Columns for function innervation, type, p_o assertion:
template_seed.update([("function" , "SC 'capable of' some %"),\
                      ("type", "SC %"),\
                      ("innervation", "SC 'has synaptic IO in region' some % SPLIT=|"),\
                      ("laterality", "SC 'has characteristic' some %"),\
                      ("adult_CNS", "SC 'part of' some %")])

# Create dataFrame for template
template_header = pd.DataFrame.from_records([template_seed])

template_header

In [ ]:
comment = "Only studied in females and not matched to EM data (Baker et al., 2022)."
xref = "FlyBase:FBrf0254226"
creator = "https://orcid.org/0000-0002-1373-1705"
date = "2025-05-06T12:00:00Z"
adult_CNS = "FBbt:00003623"
projection_neuron = "FBbt:00007392"

In [ ]:
def get_regions(cell_type):
    df = data[data['cell_type']==cell_type].reset_index(drop=True)
    region_names = []
    for c in df.columns.values:
        if df[c][0] == '1':
            region_names.append(c)
    region_fbbts = {region_ids['FBbt_id'][r]: region_ids['FBbt_name'][r] for r in region_names}
    return region_fbbts
#get_regions('AVLP_pr31')

In [ ]:
def check_function(cell_type):
    df = data[data['cell_type']==cell_type].reset_index(drop=True)
    if df['function'][0] == 'sine':
        return " It responds to male sine song (Baker et al., 2022)."
    if df['function'][0] == 'pulse':
        return " It responds to male pulse song (Baker et al., 2022)."
    if df['function'][0] == 'both':
        return " It responds to male pulse and sine song (Baker et al., 2022)."
    else:
        return ""
check_function('AVLP_pr31')

In [ ]:
def check_laterality(cell_type):
    df = data[data['cell_type']==cell_type].reset_index(drop=True)
    if df['laterality'][0] == 'ipsi':
        return (" It remains ipsilateral (Baker et al., 2022).", 'PATO:0000634')
    if df['laterality'][0] == 'axon':
        return (" It has an axon that crosses the midline (Baker et al., 2022).", 'PATO:0000618')
    if df['laterality'][0] == 'neurites':
        return (" Some neurites cross the midline (Baker et al., 2022).", 'PATO:0000618')
    else:
        return ("", '')
check_function('AVLP_pr31')

In [ ]:
def name_lister(names):
    L = ""
    if len(names) < 1:
        return False
    elif len(names) == 1:
        return names[0]
    elif len(names) > 1:
        L = names[0]
        if len(names) > 2:
            for i in names[1:-1]:
                L = L + ", " + i
        L = L + " and " + names[-1]
        return L

In [ ]:
def definition_writer(cell_type):
    regions = get_regions(cell_type)
    definition = f"Adult projection neuron of the {cell_type} group, which innervates the {name_lister(list(regions.values()))} (Baker et al., 2022)."
    definition += check_function(cell_type)
    definition += check_laterality(cell_type)[0]
    return definition
definition_writer('AVLP_pr03')

In [ ]:
template_content = pd.DataFrame()

In [ ]:
template_content = new_terms.rename(columns={'FBbt_id':'ID'}) #type_name column keeps name
template_content['Name'] = template_content['type_name'].apply(lambda x: f'adult {x} projection neuron')
template_content['Definition'] = template_content['type_name'].apply(definition_writer)
template_content['Xref_def'] = xref
template_content['created_by'] = creator
template_content['creation_date'] = date
template_content['comment'] = comment
template_content['function'] = template_content['type_name'].apply(lambda x: 'GO:0007605' if len(check_function(x))>0 else '')
template_content['laterality'] = template_content['type_name'].apply(lambda x: check_laterality(x)[1])
template_content['type'] = projection_neuron
template_content['innervation'] = template_content['type_name'].apply(lambda x: '|'.join(list(get_regions(x))))
template_content['adult_CNS'] = adult_CNS
#template_content.head()

In [ ]:
template = pd.concat([template_header, template_content])
template

In [ ]:
template.to_csv('template.tsv', sep='\t', index=False)